In [ ]:
import pandas as pd
import numpy as np
#from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import numpy as np




# Other libraries for preprocessing, evaluation, etc. (import as needed)
data = pd.read_csv(r"forestdata.csv")
#col_names = ("collector.id","c.score","l.score","rain","tree.age","surface.litter","wind.intensity","humidity","tree.density","month","time.of.day", "fire")

In [ ]:
# @title
import pandas as pd

# Create an empty DataFrame to store the rows with missing values
missing_data_df = pd.DataFrame()

# Iterate over each column of the DataFrame
for column in data.columns:
    # Find rows with missing values in the current column
    missing_rows = data[data[column].isnull()]

    # If there are missing values in the current column
    if not missing_rows.empty:
        # Add a new column to indicate the name of the column
        missing_rows['Column'] = column

        # Concatenate the missing rows to the DataFrame
        missing_data_df = pd.concat([missing_data_df, missing_rows])

# Reset the index of the resulting DataFrame
missing = missing_data_df.reset_index(drop=True)

# Display the DataFrame containing rows with missing values for each column
print(missing)


In [ ]:
# Define target column name and feature column names
target_names = "fire"
feature_names = data.columns[data.columns != target_names]

# Extract features and target variable
X = data[feature_names]
y = data[target_names]

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.drop(columns=['collector.id'], inplace=True)
X_test.drop(columns=['collector.id'], inplace=True)

In [ ]:
X_train.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
X_train["time.of.day"] = X_train["time.of.day"].replace("morni7ng", "morning")

sns.countplot(x = "time.of.day", data = X_train)
plt.title  = ("Count-Time of day")
plt.xlabel = ("time.of.day")
plt.ylabel = ("count")
plt.show

X_test["time.of.day"] = X_test["time.of.day"].replace("morni7ng", "morning")
sns.countplot(x = "time.of.day", data = X_test)
plt.title = ("count-time of day")
plt.xlabel = ("time.of.day")
plt.ylabel = ("count")
plt.show

In [ ]:
from matplotlib import pyplot as plt
X_train['tree.age'].plot(kind='hist', bins=20, title='tree.age')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
def drop_nasty_rows(feature_names, target_names, X_data, y_data):
    """ manually deciding which feature names to include is a bit strange..
    remember to include the index variable or to change which is the index variable.
    """
    result_features = X_data.copy()  # Extract features based on feature_names
    result_target = y_data.copy()  # Copy the target data

    # Define conditions for dropping rows based on multiple columns
    bad_rows = (result_features["c.score"] > 1750) | (result_features["tree.age"] > 500)
    # Modify the conditions as needed based on the actual column names in your data

    # Filter rows based on the combined conditions
    result_features = result_features[~bad_rows]
    result_target = result_target[~bad_rows]

    return result_features, result_target

X_train, y_train = drop_nasty_rows(feature_names, target_names="target", X_data=X_train, y_data=y_train)

# Reset indexes of the cleaned data
X_train_clean = X_train.reset_index(drop=True)
y_train_clean = y_train.reset_index(drop=True)

In [ ]:
from matplotlib import pyplot as plt
X_train_clean['tree.age'].plot(kind='hist', bins=20, title='tree.age')
plt.gca().spines[['top', 'right',]].set_visible(False)


In [ ]:
from sklearn.impute import SimpleImputer
numeric_columns_train = X_train_clean.select_dtypes(include=['number']).columns
categorical_columns_train = X_train_clean.select_dtypes(exclude=['number']).columns

# Impute missing values for numeric columns in training data
numeric_imputer_train = SimpleImputer(strategy='median')
X_train_clean[numeric_columns_train] = numeric_imputer_train.fit_transform(X_train_clean[numeric_columns_train])

# Impute missing values for categorical columns in training data
categorical_imputer_train = SimpleImputer(strategy='most_frequent')
X_train_clean[categorical_columns_train] = categorical_imputer_train.fit_transform(X_train_clean[categorical_columns_train])

# Identify numeric and categorical columns in test data
numeric_columns_test = X_test.select_dtypes(include=['number']).columns
categorical_columns_test = X_test.select_dtypes(exclude=['number']).columns

# Impute missing values for numeric columns in test data
numeric_imputer_test = SimpleImputer(strategy='median')
X_test[numeric_columns_test] = numeric_imputer_test.fit_transform(X_test[numeric_columns_test])

# Impute missing values for categorical columns in test data
categorical_imputer_test = SimpleImputer(strategy='most_frequent')
X_test[categorical_columns_test] = categorical_imputer_test.fit_transform(X_test[categorical_columns_test])

# Check for missing values in the imputed training set
print("Missing values in X_train after imputation:")
print(X_train_clean.isnull().sum())

# Check for missing values in the imputed test set
print("\nMissing values in X_test after imputation:")
print(X_test.isnull().sum())


# Now, X_train and X_test contain imputed values without leakage from the test set

***NORMALIZATION AND ENCODING***

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

# Define the data preparation for the columns
#
# Categorical columns will have a one hot encoder applied
#   (note - we also set this to drop the first "dummy" column, similar to what we did last week; we have also
#    told it how to handle unknown values, those that don't appear in the training data but do appear in validation
#    or testing. 'infrequent_if_exist' assigns any unknown values to a separate category)
#
# Numerical columns will be normalized
#
# Each processing step also includes the list of columns that it will be applied to
#numerical_X_train = X_train_clean.select_dtypes(include=["int64", "float64"]).columns
#categorical_X_train = X_train_clean.select_dtypes(include=["object", "bool"]).columns
numerical_X_train = ["c.score", "l.score", "rain", "tree.age", "surface.litter", "wind.intensity", "humidity", "tree.density", "month"]
categorical_X_train = ["time.of.day"]

print(numerical_X_train)
print(categorical_X_train)

t = [('cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist'), categorical_X_train), ('num', MinMaxScaler(), numerical_X_train)]
col_transform = ColumnTransformer(transformers=t)

# Fit() works out which columns to make and stores them in "enc"
col_transform.fit(X_train_clean)

# Transform() then makes those columns in the data set we provide
X_train_encod = col_transform.transform(X_train_clean)
X_test_encod = col_transform.transform(X_test)  # Transform X_test as well

# X_train_encoded and X_test_encoded will now be numpy arrays, which can be used for training and testing
# The columns in these are the transformed data
train_view = pd.DataFrame(X_train_encod)
test_view = pd.DataFrame(X_test_encod)

train_view.head(10)
test_view.head(10)


In [ ]:
#Assuming you already have split data (X_train, y_train)
from imblearn.over_sampling import SMOTE
#Apply SMOTE only to the training data
X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train_encod, y_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

# Define models with default hyperparameters
decision_tree_model = DecisionTreeClassifier(random_state=37)
logistic_regression_model = LogisticRegression(random_state=37)
neural_network_model = MLPClassifier(random_state=37)

# Define hyperparameter grids
decision_tree_params = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

logistic_regression_params = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

neural_network_params = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['logistic', 'relu'],
    'alpha': [0.0001, 0.001, 0.01]
}

# Define evaluation metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# GridSearchCV for each model with recall scoring
decision_tree_grid = GridSearchCV(decision_tree_model, decision_tree_params, scoring=scoring, refit='recall', cv=5)
logistic_regression_grid = GridSearchCV(logistic_regression_model, logistic_regression_params, scoring=scoring, refit='recall', cv=5)
neural_network_grid = GridSearchCV(neural_network_model, neural_network_params, scoring=scoring, refit='recall', cv=5)

# Fit models
decision_tree_grid.fit(X_train_resampled, y_train_resampled)
logistic_regression_grid.fit(X_train_resampled, y_train_resampled)
neural_network_grid.fit(X_train_resampled, y_train_resampled)


# Get best models
best_decision_tree_model = decision_tree_grid.best_estimator_
best_logistic_regression_model = logistic_regression_grid.best_estimator_
best_neural_network_model = neural_network_grid.best_estimator_


In [ ]:

# Get best hyperparameters from GridSearchCV (Cell 1)
best_decision_tree_params = decision_tree_grid.best_params_
best_logistic_regression_params = logistic_regression_grid.best_params_
best_neural_network_params = neural_network_grid.best_params_

# Train the models with best hyperparameters
best_decision_tree_model.fit(X_train_resampled, y_train_resampled)
best_logistic_regression_model.fit(X_train_resampled, y_train_resampled)
best_neural_network_model.fit(X_train_resampled, y_train_resampled)

decision_tree_predictions = best_decision_tree_model.predict(X_test_encod)
logistic_regression_predictions = best_logistic_regression_model.predict(X_test_encod)
neural_network_predictions = best_neural_network_model.predict(X_test_encod)

# Print training completion message (optional)
print("Final Training Completed!")


In [ ]:
 decision_tree_grid.best_params_


In [ ]:

 logistic_regression_grid.best_params_



In [ ]:

 neural_network_grid.best_params_


In [ ]:
 #Evaluate models
decision_tree_accuracy = accuracy_score(y_test, decision_tree_predictions)
logistic_regression_accuracy = accuracy_score(y_test, logistic_regression_predictions)
neural_network_accuracy = accuracy_score(y_test, neural_network_predictions)

decision_tree_precision = precision_score(y_test, decision_tree_predictions)
logistic_regression_precision = precision_score(y_test, logistic_regression_predictions)
neural_network_precision = precision_score(y_test, neural_network_predictions)

decision_tree_recall = recall_score(y_test, decision_tree_predictions)
logistic_regression_recall = recall_score(y_test, logistic_regression_predictions)
neural_network_recall = recall_score(y_test, neural_network_predictions)

decision_tree_f1 = f1_score(y_test, decision_tree_predictions)
logistic_regression_f1 = f1_score(y_test, logistic_regression_predictions)
neural_network_f1 = f1_score(y_test, neural_network_predictions)

decision_tree_auc = roc_auc_score(y_test, decision_tree_predictions)
logistic_regression_auc = roc_auc_score(y_test, logistic_regression_predictions)
neural_network_auc = roc_auc_score(y_test, neural_network_predictions)

# Print accuracies
print("Decision Tree Metrics:")
print("Accuracy:", decision_tree_accuracy)
print("Precision:", decision_tree_precision)
print("Recall:", decision_tree_recall)
print("F1 Score:", decision_tree_f1)
print("AUC:", decision_tree_auc)

print("\nLogistic Regression Metrics:")
print("Accuracy:", logistic_regression_accuracy)
print("Precision:", logistic_regression_precision)
print("Recall:", logistic_regression_recall)
print("F1 Score:", logistic_regression_f1)
print("AUC:", logistic_regression_auc)

print("\nNeural Network Metrics:")
print("Accuracy:", neural_network_accuracy)
print("Precision:", neural_network_precision)
print("Recall:", neural_network_recall)
print("F1 Score:", neural_network_f1)
print("AUC:", neural_network_auc)


In [ ]:

from sklearn import metrics

y_pred = best_neural_network_model.predict(X_test_encod)
print("r2:", metrics.r2_score(y_test, y_pred))
print("MSE:", metrics.mean_squared_error(y_test, y_pred))


# how many under 1 min, 3 mins?
#predDifferences = abs(y_pred - y_test)
#np.where(predDifferences < 3)
predDifferences = abs(np.subtract(y_pred, y_test).ravel())
print("Fraction < 1min:", len(np.where(predDifferences < 1)[0]) / len(y_test))
print("Fraction < 3min:", len(np.where(predDifferences < 3)[0]) / len(y_test))

# distributions
hist, bins = np.histogram(predDifferences, bins=range(0,15,1))
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)

In [ ]:
from sklearn.metrics import confusion_matrix

# Assuming you have true labels (y_test) and predicted labels (e.g., decision_tree_predictions)
confusion_matrix_dt = confusion_matrix(y_test, decision_tree_predictions)
confusion_matrix_lr = confusion_matrix(y_test, logistic_regression_predictions)
confusion_matrix_nn = confusion_matrix(y_test, neural_network_predictions)

# Print the confusion matrices for each model
print("Decision Tree Confusion Matrix:\n", confusion_matrix_dt)
print("Logistic Regression Confusion Matrix:\n", confusion_matrix_lr)
print("Neural Network Confusion Matrix:\n", confusion_matrix_nn)
